<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab9/In-Context-Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab9

Mounted at /content/drive
/content/drive/MyDrive/ai-intensive2
Already up to date.
/content/drive/MyDrive/ai-intensive2/lab9


In [3]:
!pip install -q transformers

# 텍스트 생성
## Greedy Search
![](https://miro.medium.com/v2/resize:fit:1400/1*Dqw1Y7zDcxKOBvsl_bbtog.png)

## Beam Search
![](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)
![](https://www.baeldung.com/wp-content/uploads/sites/4/2021/10/beamsearch.jpg)
## Sampling
![](https://huggingface.co/blog/assets/02_how-to-generate/sampling_search.png)

### Top-k Sampling
![](https://www.researchgate.net/publication/370841754/figure/fig2/AS:11431281159484568@1684378994491/Top-K-sampling-is-insensitive-to-model-confidence.png)

### Top-p Sampling (Nucleus Sampling)
![](https://miro.medium.com/v2/resize:fit:720/format:webp/0*J37qonVPJvKZpzv2)

### Temperature Sampling
![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*yJUw-zkpTfqdTVeOO7OKnQ.png)

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

# 경고 안띄우기
transformers.logging.set_verbosity_error()

In [5]:
# 생성이 오래걸리므로 가급적 GPU로 하시길 권장드립니다
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_id = "42dot/42dot_LLM-PLM-1.3B"
# model_id = "heegyu/ajoublue-gpt2-medium"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device).eval()
if "cuda" in device:
  model.half() # float16 으로 더 빠르게 생성

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/871k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/529k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.76G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

In [6]:
def generate(prefix, max_new_tokens=16, **kwargs):
  input_ids = tokenizer.encode(prefix, return_tensors="pt").to(device)
  outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, **kwargs)
  print(tokenizer.decode(outputs[0]))

In [7]:
# greedy
generate(
    "오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,\n",
    )

# beam search (beam size=4)
generate(
    "오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,\n",
    num_beams=4
    )

<||bos||>오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,
“야, 너희들, 오늘 뭐해?”
“응, 뭐
<||bos||>오늘도 무척이나 심심한 하루였다. 뭐 재밌는 일이 없을까 고민하던 찰나,
그때 문득 떠오른 아이디어가 있었다.
바로,
"오늘


In [8]:
prompt = """수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 """

generate(
  prompt,
  do_sample=True
)

generate(
  prompt,
  do_sample=True,
  temperature=0.1
)

generate(
  prompt,
  do_sample=True,
  temperature=5.0
)

<||bos||>수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ~~수능 전날 밤! ~~수능날 준비운동은 필수!
<||bos||>수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ~~수능날 긴장 안하는 꿀팁!을 알려드릴게요
<||bos||>수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ଓ의 모든지식이 동원되는 기간인 것을.. Խ


In [9]:
prompt = """수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 """

generate(
  prompt,
  do_sample=True,
  top_k=50,
  top_p=0.7
)


<||bos||>수능날 긴장 안하는 꿀팁!
고삼 여러분 수능이 얼마 남지 않았습니다. 그동안 노력한 결실을 맺기 위해서 ~~ 오늘도 파이팅입니다. 수능시험이 코앞에 다가왔어요


# In-context Learning

In [10]:
generate("""문장을 읽고 긍정/부정 중 하나를 선택하세요
이 영화는 엄복동 급의 명작입니다=""", 4)

<||bos||>문장을 읽고 긍정/부정 중 하나를 선택하세요
이 영화는 엄복동 급의 명작입니다=This movie is a


In [11]:
generate("""문장을 읽고 긍정/부정 중 하나를 선택하세요
아 이 영화 진짜 개노잼이다=부정
올해 최고의 명작=긍정
이 영화는 엄복동 급의 명작입니다=""", 2)

<||bos||>문장을 읽고 긍정/부정 중 하나를 선택하세요
아 이 영화 진짜 개노잼이다=부정
올해 최고의 명작=긍정
이 영화는 엄복동 급의 명작입니다=긍정


In [12]:
generate("""문장을 존대말로 바꾸세요.
안녕 뭐하니=안녕하세요, 뭐하고 지내세요?
잘 잤냐=잘 주무셨어요?
안졸려?=졸리지 않으세요?
밥 먹었냐=""", 8, do_sample=True)

<||bos||>문장을 존대말로 바꾸세요.
안녕 뭐하니=안녕하세요, 뭐하고 지내세요?
잘 잤냐=잘 주무셨어요?
안졸려?=졸리지 않으세요?
밥 먹었냐=밥은 드셨냐? 밥 먹


In [13]:
generate("""문장을 읽고 긍정/부정 중 하나로 분류하세요
왕노잼=부정
왕꿀잼=긍정
핵노잼=""", 2)

<||bos||>문장을 읽고 긍정/부정 중 하나로 분류하세요
왕노잼=부정
왕꿀잼=긍정
핵노잼=부정


In [14]:
generate("""제시된 단어를 보고 창의적인 식당 리뷰를 만들어보세요

벌레=음식에 벌레나왔어요 ㅡㅡ 진짜 위생관리가 어떻게된건지 ;; 절대 여기서 주문하지 마세요.
반찬=배달시켰는데 반찬 몇개가 빠졌어요 ㅠㅠ
별점 5점=음식이 진짜 맛있어요
배달=""", 64, do_sample=True)

<||bos||>제시된 단어를 보고 창의적인 식당 리뷰를 만들어보세요

벌레=음식에 벌레나왔어요 ㅡㅡ 진짜 위생관리가 어떻게된건지 ;; 절대 여기서 주문하지 마세요.
반찬=배달시켰는데 반찬 몇개가 빠졌어요 ㅠㅠ
별점 5점=음식이 진짜 맛있어요
배달=배달 왔을때 음식에 벌레가 나왔습니다. 반찬도 늦게오고 배고프던데 밥만 주네요..

위메프에서 딜 중이라는걸 보고 전화하니 배달안돼요, 전화번호 물어보니 전화기 꺼졌네요 ㅩㅩ 위메프 음식


In [15]:
generate("""손흥민에 대한 뉴스 제목을 작성하세요
뉴스 제목: 손흥민, EPL 득점왕 달성할 수 있을까? 전문가들의 예상은?
뉴스 제목: 손흥민 풀타임, 이강인 강인함…‘가나전’ 승리 가나요
뉴스 제목: 후배들을 기 살려준 손흥민 "쫄지마, 너희도 잘하는 선수야"
뉴스 제목: """, max_new_tokens=32, do_sample=True)

<||bos||>손흥민에 대한 뉴스 제목을 작성하세요
뉴스 제목: 손흥민, EPL 득점왕 달성할 수 있을까? 전문가들의 예상은?
뉴스 제목: 손흥민 풀타임, 이강인 강인함…‘가나전’ 승리 가나요
뉴스 제목: 후배들을 기 살려준 손흥민 "쫄지마, 너희도 잘하는 선수야"
뉴스 제목: 伐索: '손' - 손흥민, EPL 공동 득점왕...시즌 20호골!
오늘의 다른 검색어 -


In [16]:
generate(
    """이제부터 제가 재미있는 소설을 작성해보겠습니다.
내용: 요즘 우리 집안 분위기가 심상치 않다. """,
    max_new_tokens=512,
    do_sample=True,
    repetition_penalty=1.5
)

<||bos||>이제부터 제가 재미있는 소설을 작성해보겠습니다.
내용: 요즘 우리 집안 분위기가 심상치 않다.??? :??, 어디 나가요!!!(아버지) 어네가 말하지 마라! 아버지가 막 화낸다.(우리집 여자들ᄏᄏᄏᄏᄏᄏᄏ ) 내가 어쨌건 저떻던 당신 딸이 어떻게 된거 같냐? 안좋은 말을 많이 듣기는 했소 그래도 예전의 우리가 아녀예~ 당신이 하는 말이 모두 사실이란 말이다~~!!!! (사실 엄마 아빠에게 그렇게까지 말할 필요는 없을 거라 생각한다.)<|endoftext|>


In [17]:
generate("""노래 제목과 장르를 보고, 사람들이 좋아할 만한 노래 가사를 써주세요.

노래 제목: 그대라는 시
장르: 발라드
가사:
언제부터인지 그대를 보면
운명이라고 느꼈던 걸까
밤하늘의 별이 빛난 것처럼
오랫동안 내 곁에 있어요
그대라는 시가 난 떠오를 때마다
외워두고 싶어 그댈 기억할 수 있게
슬픈 밤이 오면 내가 그대를 지켜줄게
내 마음 들려오나요 잊지 말아요

노래 제목: 밤편지
장르: 발라드/어쿠스틱
가사:
이 밤 그날의 반딧불을 당신의
""", max_new_tokens=128, do_sample=True, repetition_penalty=1.2)

<||bos||>노래 제목과 장르를 보고, 사람들이 좋아할 만한 노래 가사를 써주세요.

노래 제목: 그대라는 시
장르: 발라드
가사:
언제부터인지 그대를 보면
운명이라고 느꼈던 걸까
밤하늘의 별이 빛난 것처럼
오랫동안 내 곁에 있어요
그대라는 시가 난 떠오를 때마다
외워두고 싶어 그댈 기억할 수 있게
슬픈 밤이 오면 내가 그대를 지켜줄게
내 마음 들려오나요 잊지 말아요

노래 제목: 밤편지
장르: 발라드/어쿠스틱
가사:
이 밤 그날의 반딧불을 당신의
별빛으로 간직해줘요..
이제 다시 오지 않는 건가요? 어쩌다<|endoftext|>


In [19]:
generate("""게시글 제목과 가격을 보고, 판매 문구를 작성해주세요

제목: 맥북 프로 17인치 팝니다
가격: 1300000원
내용: 맥북 프로 산지 1년도 안됬는데 안써서 팝니다 어저구 저쩌구

제목: 애플워치 미개봉 팔아요
가격: 300000원
내용: 애플워치 잘못샀어요 ㅠㅠ 싸게 팔아요

제목: 엄마가 안쓰는 화분 팝니다
가격: 50000원
내용: """, max_new_tokens=128, do_sample=True, repetition_penalty=1.2)

<||bos||>게시글 제목과 가격을 보고, 판매 문구를 작성해주세요

제목: 맥북 프로 17인치 팝니다
가격: 1300000원
내용: 맥북 프로 산지 1년도 안됬는데 안써서 팝니다 어저구 저쩌구

제목: 애플워치 미개봉 팔아요
가격: 300000원
내용: 애플워치 잘못샀어요 ㅠㅠ 싸게 팔아요

제목: 엄마가 안쓰는 화분 팝니다
가격: 50000원
내용: ゐー~ 어머니께서 쓰시던 식물을 다 버리셔서 내놓습니다. 이파만 있어도 되어서 가지고 가실 수 있으면 좋겠네요~~^^;<|endoftext|>


In [20]:
generate("""광고 내용을 보고 적절한 광고 문구를 만들어주세요

내용: 제주 생귤 1kg당 만원
광고 문구: 제초제를 전혀 사용하지 않고 나무에서 직접 딴 맛있는 제주 생귤 1kg 단돈 만원입니다.
내용: 하리보 포도맛 젤리
광고 문구: 아이들의 미소를 만드는 '하리보'의 포도맛 젤리! 초등학생들의 입맛을 사로잡은 그 맛, 이제 부모님들에게도 선사합니다.
내용: 원룸 청소업체
광고 문구: """, max_new_tokens=128, do_sample=True, repetition_penalty=1.2)

<||bos||>광고 내용을 보고 적절한 광고 문구를 만들어주세요

내용: 제주 생귤 1kg당 만원
광고 문구: 제초제를 전혀 사용하지 않고 나무에서 직접 딴 맛있는 제주 생귤 1kg 단돈 만원입니다.
내용: 하리보 포도맛 젤리
광고 문구: 아이들의 미소를 만드는 '하리보'의 포도맛 젤리! 초등학생들의 입맛을 사로잡은 그 맛, 이제 부모님들에게도 선사합니다.
내용: 원룸 청소업체
광고 문구: 【자취방청소서비스】 우리 집을 깨끗하게 해주는 자린이어죠? 저희에게 맡겨보세요~ *본 내용은 서울지역(서울특별시 포함)만 가능합니다*<|endoftext|>
